# Word Embeddings

## ¿Cómo representamos palabras, oraciones y significados en NLP?

## ¿Qué es una palabra?


![](img/words.png)

Cuando hablamos de palabras, podemos distinguir dos conceptos diferentes:

- **ocurrencia** (*token*) se refiere a una observación de una palabra en una cadena de texto. 

    Como hemos visto, en algunas lenguas es más o menos complejo identificar los límites de las palabras, pero en la mayoría de las lenguas occidentales y de nuestro entorno se utilizan espacios y otros signos de puntuación para delimitar las palabras.

- **tipo** (*type*) es la representación abstracta de una palabra. Cad **ocurrencia** pertenece a un **tipo** de palabra. Cuando contamos la frecuencia de las palabras de un *corpus* o colección de textos, lo que hacemos es contar el número de ocurrencias que tiene cada tipo.

In [7]:
from nltk import word_tokenize

texts = [
    """No hubo sorpresa en Bruselas. 621 votos a favor, 49 en contra (los 'remainers' británicos entre ellos) y 13 abstenciones.""",
    """MEPs ratified the Brexit Withdrawal Agreement by 621 votes to 49 following an emotional debate in Brussels.""",
    """環太平洋造山帯に属する小スンダ列島の西端に位置している。"""
]

for text in texts:
    print(word_tokenize(text))

['No', 'hubo', 'sorpresa', 'en', 'Bruselas', '.', '621', 'votos', 'a', 'favor', ',', '49', 'en', 'contra', '(', 'los', "'remainers", "'", 'británicos', 'entre', 'ellos', ')', 'y', '13', 'abstenciones', '.']
['MEPs', 'ratified', 'the', 'Brexit', 'Withdrawal', 'Agreement', 'by', '621', 'votes', 'to', '49', 'following', 'an', 'emotional', 'debate', 'in', 'Brussels', '.']
['環太平洋造山帯に属する小スンダ列島の西端に位置している。']


In [8]:
tweets = [
    """🎉¡#SORTEO! Gana una tostadora YummyToast Double. 🎁 
▪️Síguenos. 
▪️Comenta mencionando a 2 amigos junto a #Cecotec.
Tienes hasta el 9 de febrero para participar. El regalo se sorteará aleatoriamente entre los participantes. ¡Mucha suerte!.""",
    """we play for y’all 🏀‼️🖤 https://t.co/sd12vW93 #MambaMentality"""
]

for tweet in tweets:
    print(word_tokenize(tweet))

['🎉¡', '#', 'SORTEO', '!', 'Gana', 'una', 'tostadora', 'YummyToast', 'Double', '.', '🎁', '▪️Síguenos', '.', '▪️Comenta', 'mencionando', 'a', '2', 'amigos', 'junto', 'a', '#', 'Cecotec', '.', 'Tienes', 'hasta', 'el', '9', 'de', 'febrero', 'para', 'participar', '.', 'El', 'regalo', 'se', 'sorteará', 'aleatoriamente', 'entre', 'los', 'participantes', '.', '¡Mucha', 'suerte', '!', '.']
['we', 'play', 'for', 'y', '’', 'all', '🏀‼️🖤', 'https', ':', '//t.co/sd12vW93', '#', 'MambaMentality']


In [10]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

for text in texts:
    print(tokenizer.tokenize(text))
    
for tweet in tweets:
    print(tokenizer.tokenize(tweet))

['No', 'hubo', 'sorpresa', 'en', 'Bruselas', '.', '621', 'votos', 'a', 'favor', ',', '49', 'en', 'contra', '(', 'los', "'", 'remainers', "'", 'británicos', 'entre', 'ellos', ')', 'y', '13', 'abstenciones', '.']
['MEPs', 'ratified', 'the', 'Brexit', 'Withdrawal', 'Agreement', 'by', '621', 'votes', 'to', '49', 'following', 'an', 'emotional', 'debate', 'in', 'Brussels', '.']
['環太平洋造山帯に属する小スンダ列島の西端に位置している', '。']
['🎉', '¡', '#SORTEO', '!', 'Gana', 'una', 'tostadora', 'YummyToast', 'Double', '.', '🎁', '▪', '️', 'Síguenos', '.', '▪', '️', 'Comenta', 'mencionando', 'a', '2', 'amigos', 'junto', 'a', '#Cecotec', '.', 'Tienes', 'hasta', 'el', '9', 'de', 'febrero', 'para', 'participar', '.', 'El', 'regalo', 'se', 'sorteará', 'aleatoriamente', 'entre', 'los', 'participantes', '.', '¡', 'Mucha', 'suerte', '!', '.']
['we', 'play', 'for', 'y', '’', 'all', '🏀', '‼', '️', '🖤', 'https://t.co/sd12vW93', '#MambaMentality']


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

for text in texts + tweets:
    doc = nlp(text)
    tokens = [token.text for token in doc]
    print(tokens)

## Palabras como elementos discretos

La manera más sencilla de representar palabras es como una secuencia ordenada de caracteres.

Comprobar si dos palabras son idénticas es lento :-(
Las palabras también se pueden tener una representación discreta si usamos números enteros arbitrarios como identificadores únicos.
Todas las palabras ocupan lo mismo :-)
Comprobar si dos palabras son iguales es rapidísimo :-)
Estos identificadores no significan nada :-(
No hay manera de relacionar palabras similares atendiendo a su identificador :-(


> “You shall know a word by the company it keeps.”
> — John R. Firth (1957)
>
>“The meaning of a word is its use in the language (…) One cannot guess how a word functions. One has to look at its use, and learn from that.”
>— Ludwig Wittgenstein (1953)


The fact that we can analyze the use of words in language to deduce their meaning is a fundamental idea of distributional semantics called the “distributional hypothesis”. This is the inspiration behind many algorithms for learning numerical representations of words (also called word embeddings).

Dos posibles definiciones de palabra:
caso (token): cada una de las observaciones de una palabra en un texto.
tipo (type): la palabra en abstracto. 